In [ ]:
#preprocessing and training data
#Predicting length of outage

In [24]:
#import libraries and data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score

outage_df=pd.read_csv('EDAoutagedata.csv')

In [2]:
# Inspect data
print(outage_df.head())
print(outage_df.info())

                Event Description  Year Date Event Began Time Event Began  \
0  Severe Weather - Thunderstorms  2014       2014-06-30         20:00:00   
1  Severe Weather - Thunderstorms  2014       2014-06-30         23:20:00   
2  Severe Weather - Thunderstorms  2014       2014-06-30         17:55:00   
3     Physical Attack - Vandalism  2014       2014-06-24         14:54:00   
4     Physical Attack - Vandalism  2014       2014-06-19         08:47:00   

  Date of Restoration Time of Restoration  \
0          2014-07-02            18:30:00   
1          2014-07-01            17:00:00   
2          2014-07-01            02:53:00   
3          2014-06-24            14:55:00   
4          2014-06-19            08:48:00   

                                Respondent       Geographic Areas NERC Region  \
0                 Exelon Corporation/ComEd               Illinois         RFC   
1  Northern Indiana Public Service Company  North Central Indiana         RFC   
2                      

In [3]:
# Define features and target
X = outage_df[['Event Description', 'Year', 'Respondent']]
y = outage_df['Duration (Hours)']


In [9]:
print(outage_df['Duration (Hours)'].isna().sum())

49


In [10]:
# Drop rows where target variable is missing
df_cleaned = outage_df.dropna(subset=['Duration (Hours)'])

# Redefine features and target
X = df_cleaned[['Event Description', 'Year', 'Respondent']]
y = df_cleaned['Duration (Hours)']

In [11]:
from sklearn.impute import SimpleImputer

# Create an imputer object with a median strategy
imputer = SimpleImputer(strategy='median')

# Reshape y to a 2D array
y = y.values.reshape(-1, 1)

# Fit and transform y
y_imputed = imputer.fit_transform(y).ravel()

# Redefine y with imputed values
y = y_imputed


In [12]:
# Preprocessing for text data
text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(stop_words='english'))
])


In [13]:
# Preprocessing for numerical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Add imputation if needed
    ('scaler', StandardScaler())
])

In [14]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Add imputation if needed
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [15]:
# Combine all transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, 'Event Description'),
        ('num', numeric_transformer, ['Year']),
        ('cat', categorical_transformer, ['Respondent'])
    ]
)

In [20]:
# Create a pipeline with preprocessing and model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [21]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('tfidf',
                                                                   TfidfVectorizer(stop_words='english'))]),
                                                  'Event Description'),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Respondent'])])),
                ('regressor', RandomForestRegressor())])

In [22]:
# Predict on test set
y_pred = model.predict(X_test)

In [25]:
# Evaluate the model
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'R^2 Score: {r2_score(y_test, y_pred)}')

Mean Squared Error: 4660.68632394693
R^2 Score: 0.5035989143421784


In [26]:
# Save the updated dataframe to a new CSV file
outage_df.to_csv('Trainingdata.csv', index=False)